### Imports

In [53]:
from utils import VerbParser, NounParser
import pandas as pd
import re
import os

### Loading data

In [54]:
pd.set_option('display.max_rows', None)
path = r"D:\ERC_Bang\Tasks\data_cleaning" 
os.chdir(path)
data = pd.read_csv("data.tsv", sep="\t", encoding="utf-8")

### Nouns

In [86]:
noun_parser=NounParser()
verb_parser=VerbParser()

In [79]:
data["PARSED"] = data[data["SINGULAR"].notna()]["SINGULAR"].apply(
    lambda x: noun_parser.first_parse_durationals(x)
).apply(lambda x: noun_parser.second_parse_durationals(x)).apply(
    lambda x: noun_parser.cvcv_segmentation(x)
).apply(
    lambda x: noun_parser.nasalized_stops(x)
).apply(
lambda x: noun_parser.hyphen_space(x)
).apply(lambda x: noun_parser.identified_suffixes(x))

In [83]:
# data.to_csv("first_noun_parse.csv", index=False)

In [85]:
data["PARSED"].head(30)

0             jáw-dì-m
1              dáábà-m
2             àrsɛ̌ɛ-m
3             kɔ́-mbɔ̀
4             bɛ́lɛ̀-g
5            bɛ́lɛ̀-gù
6                bɛ̀lú
7                bɛ̀lú
8          gàr sɛ̀-gɛ́
9            à sɛ̀-gɛ́
10          gà sɛ̀-gɛ́
11            à sɛ̀-gɛ
12          dèbù bɛ́lí
13               bɛ́lí
14           dúmɛ́-ŋgó
15            dùmɔ́ɔ-n
16          gà sɛ̀-gɛ́
17               dáábá
18           à sɛ̀-gɛ́
19           à sɛ̀-gɛ́
20          gà sɛ̀-gɛ́
21                 bèl
22               bɛ̀lì
23    gìrⁿí árúsɛ̀-gɛ̀
24       gìnɛ̀-ý bɛ̀lù
25               bɛ̀lú
26              dáábáá
27            àrùzà-ká
28             àrzà-gá
29           à zɛ̀-gɛ̀
Name: PARSED, dtype: object

### Parsing nouns

### Verbs

In [62]:
verbs = data[data["FORM"].notna()]
verbs=verbs[["VALUE_ORG", "FORM"]]
verbs=verbs["FORM"]

In [63]:
verb_parser=VerbParser()

In [64]:
verbs.apply(
    lambda x: verb_parser.parse_durationals(x)
).apply(
    lambda x: verb_parser.segment_cvcs(x))[:10]

424     pánd-í
425         pó
426      pál-í
427      úr-ɔ́
428    ʔɔ́r-ɛ̀
429    kɛ̀s-ɛ́
430      úr-ɛ̀
431      úr-ɛ̀
432     pɔ́-ɔ́
433      pán-á
Name: FORM, dtype: object